In [ ]:
import os

base_dir = './datasets'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

train_0_front_dir = os.path.join(train_dir, '0_front')
train_1_back_dir = os.path.join(train_dir, '1_back')
train_1_front_dir = os.path.join(train_dir, '1_front')
train_2_back_dir = os.path.join(train_dir, '2_back')
train_2_front_dir = os.path.join(train_dir, '2_front')
train_5_front_dir = os.path.join(train_dir, '5_front')
train_ILU_dir = os.path.join(train_dir, 'ILU')

val_0_front_dir = os.path.join(val_dir, '0_front')
val_1_back_dir = os.path.join(val_dir, '1_back')
val_1_front_dir = os.path.join(val_dir, '1_front')
val_2_back_dir = os.path.join(val_dir, '2_back')
val_2_front_dir = os.path.join(val_dir, '2_front')
val_5_front_dir = os.path.join(val_dir, '5_front')
val_ILU_dir = os.path.join(val_dir, 'ILU')

save_dir = os.path.join(base_dir, 'saved_models')

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers

# batch_norm(batch_re-norm), ensemble, regularization 안했음

h = 150 # height
w = 150 # width
c = 3  # channels
num_classes = 7

model = models.Sequential()

model.add(layers.Conv2D(128, 3, activation='relu', input_shape=(h, w, c,)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, 3, activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(512, 3, activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.4, 
                                   height_shift_range=0.4,
                                   shear_range=0.4,
                                   zoom_range=[1, 2],
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=25)

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(150, 150),
                                                batch_size=25)

In [ ]:
import numpy as np

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop

callbacks_list = [
    EarlyStopping(
#         monitor='val_loss',
        monitor='val_acc',
        patience=10
    ),
    ModelCheckpoint(
        filepath=os.path.join(save_dir, 'hand_sign.h5'),
#         monitor='val_loss',
        monitor='val_acc',
        save_best_only=True
    ),
#     ReduceLROnPlateau(
#         # 검증 손실이 10 epoch동안 좋아지지 않으면 학습률에 0.1을 곱함(학습률을 1/10으로)
#         monitor='val_loss',
#         factor=0.1,
#         patience=10
#     )
]

model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['acc'])

steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
validation_steps = np.ceil(val_generator.samples/val_generator.batch_size)

history = model.fit(train_generator, epochs=100,
                    steps_per_epoch=steps_per_epoch,
                    callbacks=callbacks_list,
                    validation_data=val_generator,
                    validation_steps=validation_steps)

model.save(os.path.join(save_dir, 'hand_sign_2.h5'))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation_loss')
plt.title('Loss')
plt.legend()

plt.show()

# # 그래프가 너무 지그재그면 하기
# def smooth_curve(points, factor=0.9):
#     smoothed_curve = []
#     for point in points:
#         if smoothed_curve:
#             prev = smoothed_curve[-1]
#             smoothed_curve.append(prev*factor + point*(1-factor))
#         else:
#             smoothed_curve.append(point)
            
#     return smoothed_curve

# plt.plot(epochs, smooth_curve(acc), 'b', label='smoothed train_acc')
# plt.plot(epochs, smooth_curve(val_acc), 'r', label='smoothed val_acc')
# plt.title("Accuracy")
# plt.legend()
# plt.figure()

# plt.plot(epochs, smooth_curve(loss), 'b', label='smoothed train_loss')
# plt.plot(epochs, smooth_curve(val_loss), 'r', label='smoothed val_loss')
# plt.title("Loss")
# plt.legend()
# plt.show()